In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="report-udacity1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: ws-udacity
Azure region: japaneast
Subscription id: 8d1a13c1-dda4-4fdf-a927-e08a4213f4e3
Resource group: resoure-udacity


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
cluster_name = "udacity-cluster"

#: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
vm_size = "Standard_D2_V2"
# Check if the cluster already exists, and create it if it doesn't
try:
    compute_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Using existing compute cluster.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)


Using existing compute cluster.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(parameter_space={
    '--C': uniform(0.1, 100),
    '--max_iter' : choice(10,20,30)})

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 1, delay_evaluation= 5 )

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=".",
                      script="train.py",
                      compute_target=cluster_name,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config = src, 
    hyperparameter_sampling = ps,
    policy = policy,
    primary_metric_name = "Accuracy",
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
    max_total_runs = 6,
    max_concurrent_runs = 4
    )

In [28]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
from azureml.widgets import RunDetails
hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)



In [35]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

4

In [34]:
import joblib
# Get your best run and save the model from that run.
model_index=best_run.get_file_names().index('outputs/model.joblib')
best_run.download_file(best_run.get_file_names()[model_index], output_file_path='./outputs/')

### YOUR CODE HERE ###
joblib.load('./outputs/model.joblib')


Trying to unpickle estimator LogisticRegression from version 0.24.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.


LogisticRegression(C=51.486246965511484, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=10, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [36]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [38]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
alldata=x.join(y)

In [39]:
alldata

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,1,0,0,1,7,1,116,1,999,...,0,1,0,0,0,0,0,0,0,0
32946,37,1,0,0,1,7,5,69,7,999,...,0,0,0,0,0,0,0,1,0,0
32947,26,0,0,0,0,5,2,135,4,999,...,0,0,0,0,0,0,0,1,0,0
32948,31,0,0,0,0,4,1,386,1,999,...,0,0,0,1,0,0,0,0,0,0


In [40]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=alldata,
    label_column_name='y',
    n_cross_validations=2)

In [41]:
# Submit your automl run

### YOUR CODE HERE ###
remote_run = exp.submit(automl_config, show_output = False)

2023-07-02:20:52:40,478 WARNING  [local_experiment_driver.py:142] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
report-udacity1,AutoML_5910ac6f-4a34-4825-823a-f032c92ebe01,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2023/07/02 21:00:47 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [43]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, best_model = remote_run.get_output()
best_run.register_model(model_name = 'automl_best_model.pkl', model_path = './outputs/')

Model(workspace=Workspace.create(name='ws-udacity', subscription_id='8d1a13c1-dda4-4fdf-a927-e08a4213f4e3', resource_group='resoure-udacity'), name=automl_best_model.pkl, id=automl_best_model.pkl:2, version=2, tags={}, properties={})